 # Install/Import Packages and Download/Unzip Dataset

 ---

 The following code installs d3rlpy, imports all necessary dependencies, and downloads/unzips the dataset from Zenodo.

In [ ]:
!pip install d3rlpy==2.6.1 gymnasium==0.29.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.7/721.7 kB 4.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 186.1/186.1 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.2/67.2 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.7 MB/s eta 0:00:00
  Created wheel for gym: filename=gym-0.26.2-py3-none-any.whl size=827620 sha256=6cce390a1418af14d2bfb031fdd1d313c9cdfe7d43c513fdf7d43006150b6451
  Stored in directory: /root/.cache/pip/wheels/b9/22/6d/3e7b32d98451b4cd9d12417052affbeeeea012955d437da1da
Successfully built gym
  Attempting uninstall: gym
    Found existing installation: gym 0.25.2
    Uninstalling gym-0.25.2:
      Successfully uninstalled gym-0.25.2
ERROR: pip's dependency resolver does not curr

In [ ]:
from google.colab import files
import numpy as np
import d3rlpy
import os

In [ ]:
!wget -O full_dataset.zip "https://zenodo.org/record/13830810/files/full_dataset.zip?download=1"

--2024-09-25 22:09:05--  https://zenodo.org/record/13830810/files/full_dataset.zip?download=1
Resolving zenodo.org (zenodo.org)... 188.184.98.238, 188.184.103.159, 188.185.79.172, ...
Connecting to zenodo.org (zenodo.org)|188.184.98.238|:443... connected.
HTTP request sent, awaiting response... 301 MOVED PERMANENTLY
Location: /records/13830810/files/full_dataset.zip [following]
--2024-09-25 22:09:06--  https://zenodo.org/records/13830810/files/full_dataset.zip
Reusing existing connection to zenodo.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 2161582746 (2.0G) [application/octet-stream]
Saving to: ‘full_dataset.zip’

full_dataset.zip    100%[===================>]   2.01G  27.6MB/s    in 78s     

2024-09-25 22:10:24 (26.5 MB/s) - ‘full_dataset.zip’ saved [2161582746/2161582746]



In [ ]:
!unzip full_dataset.zip -d /content/
data_dir = '/content/content/data/data'

Streaming output truncated to the last 5000 lines.
  inflating: /content/content/data/data/ep_38406.npz  
  inflating: /content/content/data/data/ep_45886.npz  
  inflating: /content/content/data/data/ep_80720.npz  
  inflating: /content/content/data/data/ep_41811.npz  
  inflating: /content/content/data/data/ep_74361.npz  
  inflating: /content/content/data/data/ep_36142.npz  
  inflating: /content/content/data/data/ep_92647.npz  
  inflating: /content/content/data/data/ep_35769.npz  
  inflating: /content/content/data/data/ep_70452.npz  
  inflating: /content/content/data/data/ep_89001.npz  
  inflating: /content/content/data/data/ep_92324.npz  
  inflating: /content/content/data/data/ep_21612.npz  
  inflating: /content/content/data/data/ep_51788.npz  
  inflating: /content/content/data/data/ep_51083.npz  
  inflating: /content/content/data/data/ep_54442.npz  
  inflating: /content/content/data/data/ep_97628.npz  
  inflating: /content/content/data/data/ep_26720.npz  
  inflating: /

# Prepare Dataset for d3rlpy

---

The following code loads all of the data into arrays and creates a d3rlpy dataset.

In [ ]:
total_steps = 5000000

all_observations = np.empty((total_steps, 3, 64, 64), dtype=np.uint8)
all_actions = np.empty((total_steps, 3), dtype=np.float32)
all_rewards = np.empty((total_steps,), dtype=np.float32)
all_terminals = np.empty((total_steps,), dtype=bool)

current_index = 0

for i in range(1, 1001):
    episode_file = os.path.join(data_dir, f'ep_{i}.npz')
    episode_data = np.load(episode_file)

    num_steps = episode_data['terminals'].shape[0]

    all_observations[current_index:current_index + num_steps] = np.transpose(episode_data['observations'], (0, 3, 1, 2))
    all_actions[current_index:current_index + num_steps] = episode_data['actions']
    all_rewards[current_index:current_index + num_steps] = episode_data['rewards']
    all_terminals[current_index:current_index + num_steps] = episode_data['terminals']

    current_index += num_steps

all_observations = all_observations[:current_index]
all_actions = all_actions[:current_index]
all_rewards = all_rewards[:current_index]
all_terminals = all_terminals[:current_index]

In [ ]:
dataset = d3rlpy.dataset.MDPDataset(
    observations=all_observations,
    actions=all_actions,
    rewards=all_rewards,
    terminals=all_terminals,
)

2024-09-25 22:15.44 [info     ] Signatures have been automatically determined. action_signature=Signature(dtype=[dtype('float32')], shape=[(3,)]) observation_signature=Signature(dtype=[dtype('uint8')], shape=[(3, 64, 64)]) reward_signature=Signature(dtype=[dtype('float32')], shape=[(1,)])
2024-09-25 22:15.44 [info     ] Action-space has been automatically determined. action_space=<ActionSpace.CONTINUOUS: 1>
2024-09-25 22:15.44 [info     ] Action size has been automatically determined. action_size=3


# Model Training

---

The following code trains a behavior cloning algorithm for 50 epochs at 10000 steps per epoch.

In [ ]:
BC = d3rlpy.algos.BCConfig(observation_scaler=d3rlpy.preprocessing.PixelObservationScaler(),).create(device='cuda')
BC.fit(dataset=dataset, n_steps=500000, n_steps_per_epoch=10000)

2024-09-25 22:15.44 [info     ] dataset info                   dataset_info=DatasetInfo(observation_signature=Signature(dtype=[dtype('uint8')], shape=[(3, 64, 64)]), action_signature=Signature(dtype=[dtype('float32')], shape=[(3,)]), reward_signature=Signature(dtype=[dtype('float32')], shape=[(1,)]), action_space=<ActionSpace.CONTINUOUS: 1>, action_size=3)
2024-09-25 22:15.44 [info     ] Directory is created at d3rlpy_logs/BC_20240925221544
2024-09-25 22:15.44 [debug    ] Building models...            
2024-09-25 22:15.45 [debug    ] Models have been built.       
2024-09-25 22:15.45 [info     ] Parameters                     params={'observation_shape': [3, 64, 64], 'action_size': 3, 'config': {'type': 'bc', 'params': {'batch_size': 100, 'gamma': 0.99, 'observation_scaler': {'type': 'pixel', 'params': {}}, 'action_scaler': {'type': 'none', 'params': {}}, 'reward_scaler': {'type': 'none', 'params': {}}, 'learning_rate': 0.001, 'policy_type': 'deterministic', 'optim_factory': {'type': '

Epoch 1/50:   0%|          | 0/10000 [00:00<?, ?it/s]

2024-09-25 22:17.09 [info     ] BC_20240925221544: epoch=1 step=10000 epoch=1 metrics={'time_sample_batch': 0.0030334354162216185, 'time_algorithm_update': 0.005135714602470398, 'loss': 0.006612926368432818, 'time_step': 0.008275382828712463} step=10000
2024-09-25 22:17.09 [info     ] Model parameters are saved to d3rlpy_logs/BC_20240925221544/model_10000.d3


Epoch 2/50:   0%|          | 0/10000 [00:00<?, ?it/s]

2024-09-25 22:18.32 [info     ] BC_20240925221544: epoch=2 step=20000 epoch=2 metrics={'time_sample_batch': 0.0030843403816223146, 'time_algorithm_update': 0.005055114841461181, 'loss': 0.0008610126124694944, 'time_step': 0.00823603835105896} step=20000
2024-09-25 22:18.32 [info     ] Model parameters are saved to d3rlpy_logs/BC_20240925221544/model_20000.d3


...

Epoch 49/50:   0%|          | 0/10000 [00:00<?, ?it/s]

2024-09-25 23:22.51 [info     ] BC_20240925221544: epoch=49 step=490000 epoch=49 metrics={'time_sample_batch': 0.0030454928636550905, 'time_algorithm_update': 0.005028397107124329, 'loss': 5.199615664441808e-05, 'time_step': 0.008181882309913636} step=490000
2024-09-25 23:22.51 [info     ] Model parameters are saved to d3rlpy_logs/BC_20240925221544/model_490000.d3


Epoch 50/50:   0%|          | 0/10000 [00:00<?, ?it/s]

2024-09-25 23:24.13 [info     ] BC_20240925221544: epoch=50 step=500000 epoch=50 metrics={'time_sample_batch': 0.0030592368125915525, 'time_algorithm_update': 0.005022936320304871, 'loss': 6.158876042991323e-05, 'time_step': 0.00819354944229126} step=500000
2024-09-25 23:24.13 [info     ] Model parameters are saved to d3rlpy_logs/BC_20240925221544/model_500000.d3


[(1,
  {'time_sample_batch': 0.0030334354162216185,
   'time_algorithm_update': 0.005135714602470398,
   'loss': 0.006612926368432818,
   'time_step': 0.008275382828712463}),
 (2,
  {'time_sample_batch': 0.0030843403816223146,
   'time_algorithm_update': 0.005055114841461181,
   'loss': 0.0008610126124694944,
   'time_step': 0.00823603835105896}),
 (3,
  {'time_sample_batch': 0.0030408738136291505,
   'time_algorithm_update': 0.005010224819183349,
   'loss': 0.00048361232985334935,
   'time_step': 0.00814808669090271}),
 (4,
  {'time_sample_batch': 0.0030202298879623414,
   'time_algorithm_update': 0.0049900271892547605,
   'loss': 0.00033574361195642266,
   'time_step': 0.008103820538520812}),
 (5,
  {'time_sample_batch': 0.0029990467071533203,
   'time_algorithm_update': 0.004988898253440857,
   'loss': 0.0002664233852759935,
   'time_step': 0.00808110785484314}),
 (6,
  {'time_sample_batch': 0.0029978957176208494,
   'time_algorithm_update': 0.004985409140586853,
   'loss': 0.000218

In [ ]:
!zip -r model_hundredth_dataset.zip d3rlpy_logs

  adding: d3rlpy_logs/ (stored 0%)
  adding: d3rlpy_logs/BC_20240925221544/ (stored 0%)
  adding: d3rlpy_logs/BC_20240925221544/model_90000.d3 (deflated 39%)
  adding: d3rlpy_logs/BC_20240925221544/time_algorithm_update.csv (deflated 61%)
  adding: d3rlpy_logs/BC_20240925221544/time_step.csv (deflated 60%)
  adding: d3rlpy_logs/BC_20240925221544/model_30000.d3 (deflated 23%)
  adding: d3rlpy_logs/BC_20240925221544/model_320000.d3 (deflated 44%)
  adding: d3rlpy_logs/BC_20240925221544/model_440000.d3 (deflated 45%)
  adding: d3rlpy_logs/BC_20240925221544/model_340000.d3 (deflated 44%)
  adding: d3rlpy_logs/BC_20240925221544/model_250000.d3 (deflated 44%)
  adding: d3rlpy_logs/BC_20240925221544/model_160000.d3 (deflated 42%)
  adding: d3rlpy_logs/BC_20240925221544/model_290000.d3 (deflated 44%)
  adding: d3rlpy_logs/BC_20240925221544/model_130000.d3 (deflated 41%)
  adding: d3rlpy_logs/BC_20240925221544/model_500000.d3 (deflated 45%)
  adding: d3rlpy_logs/BC_20240925221544/model_140000.d